# NLR biology ワークショップ (2024 冬)

# バイオインフォマティクス実習

# FASTAファイルを自由に使い回す方法について
　アセンブルをすると、復元されたゲノム配列のデータが手に入ります。ゲノム配列は基本的に**FASTA**ファイルと呼ばれる形式で保存されています。

　リードのデータは**FASTQ**と呼ばれる４行で１つのリード情報を示す形式でファイルに保存されていましたが、FASTA形式では、シンプルに2行で1つの配列データの情報を表します。

```
>配列の名前 (名前の後にスペースを空けて色々情報が書いてあることも)
TGAAGCAGT..............(リードの塩基配列)
```

言ってしまえばFASTAファイルもテキストファイルなので、コピーペーストなど普通のテキストファイルの様に扱うことは出来ますが、コマンドや`seqkit`等のツールを使用することで、より効率的にFASTAファイルを扱うことが出来ます。

アセンブルをしている間、このFASTAファイルを使うための方法を学びましょう。




# 今回の実習内容

今回はいくつかのコマンド、`seqkit`と`blast`というツールを使って以下の様な実習を行います。

1. 必要な情報をFASTAから抜き出す(seqkit)
1. 遺伝子Aと相同性のある配列をFASTAから抜き出す(blast, seqkit)
1. 指定領域の配列をFASTAから抜き出す(seqkit)
1. seqkitの他の機能





### データのダウンロード
　この実習で使用するサンプルデータをダウンロードします。

ここでは公開されているゲノム配列を使用します。

[Ensemble](https://asia.ensembl.org/info/data/ftp/index.html)や[NCBI](https://ncbi.nlm.nih.gov/datasets/genome/)のサーバーにあるデータは、`wget`コマンドでダウンロードできます。
```
wget ダウンロードリンク
```

例として[イネいもち病菌のゲノム配列](https://ftp.ensemblgenomes.ebi.ac.uk/pub/fungi/release-60/fasta/magnaporthe_oryzae/)をダウンロードするコマンドが表示されていますが、

同様のアプローチで自分の興味のある生物の配列をダウンロードすることが可能です。

In [ ]:
# ダウンロードするコマンド
# DNA配列のダウンロード
!wget https://ftp.ensemblgenomes.ebi.ac.uk/pub/fungi/release-60/fasta/magnaporthe_oryzae/dna/Magnaporthe_oryzae.MG8.dna.toplevel.fa.gz
# 各遺伝子のアミノ酸配列のダウンロード
!wget https://ftp.ensemblgenomes.ebi.ac.uk/pub/fungi/release-60/fasta/magnaporthe_oryzae/pep/Magnaporthe_oryzae.MG8.pep.all.fa.gz

配列データはサイズが大きいので大抵圧縮されてgzip形式になっています。

`gunzip`というコマンドで解凍が可能です。

In [ ]:
# 解凍
!gunzip Magnaporthe_oryzae.MG8.dna.toplevel.fa.gz
!gunzip Magnaporthe_oryzae.MG8.pep.all.fa.gz

ファイルをメモ帳などで開いて中身を見ることも当然可能ですが、

`head`というコマンドを使うことで、ファイルの頭からn行目までを表示させることが出来ます。

(指定しなければ10行)

まずは先ほどダウンロードしたファイルの最初の何行かを見てみましょう。

In [ ]:
!head -n 5 Magnaporthe_oryzae.MG8.dna.toplevel.fa

また、`mv 元のファイル名 新しいファイル名`というコマンドでファイル名を変更することも出来ます。

ファイル名が長くて以降の実習でややこしくなるので、ゲノム配列を`genome.fasta`、アミノ酸配列を`protein.fasta`に変更しておきます。

In [ ]:
!mv Magnaporthe_oryzae.MG8.dna.toplevel.fa genome.fasta
!mv Magnaporthe_oryzae.MG8.pep.all.fa protein.fasta

これで、イネいもち病菌のゲノム配列(genome.fasta)と全遺伝子のアミノ酸配列(protein.fasta)が準備出来ました。

**※色々コマンドが出てきていますが、覚える必要は無く、そういうことが出来ると知っておけばOKです。**

### ソフトウェアのダウンロード・インストール

　次に、今回使用するソフトウェアのインストールをします。

本実習では、`seqkit`と`blast+`を使います。使い方はまた後程順に説明していきます。

今回はこちらでインストールするコマンドを準備しましたが、今後皆さんが自分のPC等の環境で行う際には、

マニュアルに書いてあるインストール方法を参考にしつつインストールしてみて下さい。

* `seqkit` ... https://bioinf.shenwei.me/seqkit/

* `blast+` ... https://www.ncbi.nlm.nih.gov/books/NBK279690/

In [ ]:
%%bash
wget -q -O 'get_software2.sh' https://github.com/slt666666/NLR_biology_workshop_2024/raw/master/script/get_software2.sh
bash get_software2.sh
ls tools/

---
## 1. 必要な情報をFASTAから抜き出す(seqkit)

特定の遺伝子のCDS配列やアミノ酸配列が欲しい場合、

素直にやるなら、アミノ酸配列のFASTAファイルから目的の遺伝子IDを検索して配列をコピーする、という形で問題無く行えると思います。

しかし、複数の遺伝子の配列を取り出したい時や、特定の染色体の配列など長い配列をコピーする必要がある場合には少し手間になってきます。

このような場合、`seqkit`を使用することで簡単に目的の配列を取り出すことが出来ます。

<br>

**課題: MGG_01000T0という遺伝子のアミノ酸配列を抜き出す。**

まずは１つの遺伝子配列を取り出してみましょう。

```
# コマンド凡例
seqkit faidx FASTAファイル名 遺伝子名
```
このコマンドでFASTAファイルから遺伝子名の配列を取り出せます。


In [ ]:
!seqkit faidx protein.fasta MGG_01000T0

コマンドを動かした出力結果は、コマンドの最後に`「>」`を使ってファイルに書き出すことが出来ます。

```
# コマンド凡例
seqkit faidx FASTAファイル名 遺伝子名 > 任意のファイル名
```

In [ ]:
!seqkit faidx protein.fasta MGG_01000T0 > MGG_01000T0.fasta

また、通常だと長い配列は改行して出力されますが、`-w 0`というオプションを付けると、改行を無くして出力する事も出来ます。

```
# コマンド凡例
seqkit faidx -w 0 FASTAファイル名 遺伝子名 > 任意のファイル名
```

In [ ]:
!seqkit faidx -w 0 protein.fasta MGG_01000T0 > MGG_01000T0.fasta

**練習: 3という配列(3番染色体の配列)をgenome.fastaから取り出してファイルに書き出してみましょう**

XXXの部分を適宜書き換えてみてください。

In [ ]:
!seqkit faidx XXX XXX > chr3.fasta

**課題: 複数の遺伝子のアミノ酸配列をまとめて抜き出す**

次は、

* MGG_01000T0
* MGG_02000T0
* MGG_03000T0
* MGG_04000T0
* MGG_05000T0

の5つの配列を取り出してみます。

抜き出したい遺伝子のリストを書いたテキストファイルを作成して、下記のコマンドで抜き出すことが出来ます。

今回はあらかじめlist.txtというファイルを作成してあるので、これを使ってみてください。

```
# コマンド凡例
seqkit faidx -w 0 FASTAファイル名 -l 抜き出したい遺伝子リスト > 任意のファイル名
```

In [ ]:
!seqkit faidx -w 0 protein.fasta -l list.txt > Pickup.fasta

**練習: 1,2,3という配列(1~3番染色体の配列)をgenome.fastaから取り出してファイルに書き出してみましょう**

下記コマンドは、genome.fastaからlist.txtに書かれた配列を取り出すコマンドになります。

list.txtを適切なリストに書き換えて動かしてみてください。

In [ ]:
!seqkit faidx -w 0 genome.fasta -l list.txt > chr1_2_3.fasta

## 2. 遺伝子Aと相同性のある配列をFASTAから抜き出す(blast, seqkit)

Blastは[webサイト](https://blast.ncbi.nlm.nih.gov/Blast.cgi)で行うことが多いかもしれませんが、ツールとして活用することも可能です。(Local blastと言ったりします。)

[NCBIのblast](https://blast.ncbi.nlm.nih.gov/Blast.cgi)では、NCBIのデータベースに対して相同性の高い配列を探すことが出来ます。

<br>

一方、ツールとしてBlastを使うことで、任意の配列データの中から、相同性の高い配列を探すことが出来ます。

今回は、Avr-Pikというエフェクターをコードする遺伝子が、イネいもち菌のどの染色体にあるか探してみましょう。

AVR-Pikの配列は[ここ](https://www.ncbi.nlm.nih.gov/nuccore/AB498879)のものを利用しました([Yoshida et al., 2009](https://doi.org/10.1105/tpc.109.066324))。

<br>

Local blastを使う際には、まず検索対象の配列のデータベースを作成します。

```
# コマンド凡例
makeblastdb -in FASTAファイル名 -dbtype nucl -hash_index
```

今回は`genome.fasta`からデータベースを作成する形です。




In [ ]:
!makeblastdb -in genome.fasta -dbtype nucl -hash_index

続いて、blastを実行します。塩基配列を検索するので`blastn`を使用します。

```
# コマンド凡例
blastn -db 先ほど作成したデータベース名(FASTAファイル名) -query 検索したい配列のFASTAファイル -out 任意のファイル名.txt
```

In [ ]:
!blastn -db genome.fasta -query AVR_Pik_CDS.fasta -out blastn_result.txt

デフォルトの設定だとblast結果が見にくいので、`-outfmt`というオプションを使うことで、結果のフォーマットを変えることも出来ます。

また、任意のe-valueを`-evalue`というオプションで設定して、相同性の高さの基準を決めることも可能です。

```
# コマンド凡例
blastn -db 先ほど作成したデータベース名(FASTAファイル名) -query 検索した配列のFASTAファイル -evalue 1e-30 -outfmt 6 -out 任意のファイル名.txt
```

In [ ]:
!blastn -db genome.fasta -query AVR_Pik_CDS.fasta -evalue 1e-30 -outfmt 6 -out blastn_result_tab.txt

各列の情報は検索した配列の名前、ヒットした配列の名前、一致した位置の割合、重なった長さ...等を表しています。

参考: https://www.metagenomics.wiki/tools/blast/blastn-output-format-6

**練習: アミノ酸配列の中からAVR-Pikの配列と相同性の高いものを探してみましょう**

まず、1行目でアミノ酸配列のデータベースを作ります。<small>※アミノ酸配列なので -dbtypeがnuclからproになります。</small>

先ほどと違って塩基配列ではなく、アミノ酸配列で検索するので`blastn`ではなく`blastp`になります。

XXXの部分を書き換えて動かしてみてください。


In [ ]:
!makeblastdb -in protein.fasta -dbtype prot -hash_index

!blastp -db XXX -query XXX -outfmt 6 -out blastp_result_tab.txt

結果を見ると、いくつか似ているアミノ酸をコードする遺伝子が見つかりました。

少しややこしいので詳細は省きますが、`awk`というコマンドを使うと、`2列目だけ抜き出す`みたいなことが出来ます。

```
# コマンド凡例
awk '{print $2}' tab区切りBlast結果.txt | uniq > 任意のファイル名.txt
```

このコマンドと先ほど扱った`seqkit`を組み合わせることで、

`blastでヒットした遺伝子の配列を取り出す。`といった作業をサッと行うことが出来ます。

In [ ]:
# blast結果から2列目のみ取り出す
!awk '{print $2}' blastp_result_tab.txt | sort | uniq > blast_gene_list.txt
# seqkitでblastの結果で出た名前の遺伝子の配列を取り出す
!seqkit faidx -w 0 protein.fasta -l blast_gene_list.txt > blast_gene_list.fasta

## 指定領域の配列をFASTAから抜き出す(seqkit)

`seqkit`を使うと、領域を指定して配列を取得する事も出来ます。

```
# コマンド凡例
seqkit faidx -w 0 FASTAファイル名 配列名:start-end > 任意のファイル名
```

先ほどAVR-Pikの配列でblastをした結果、2という配列(2番染色体)の 8229912-8229571がヒットしたかと思いますが、この領域の配列を取得してみましょう。

In [ ]:
!seqkit faidx -w 0 genome.fasta 2:8229912-8229571 > AVR_Pik_candidate.fasta

**練習: このAVR-Pikのプロモーター候補領域として、上記の領域の3000bp手前の領域の配列を取得してみましょう。**

XXXの値を書き換えて動かしてみてください。

In [ ]:
!seqkit faidx -w 0 genome.fasta 2:XXX-XXX > AVR_Pik_promoter_candidate.fasta

## seqkitで出来る他のこと

`seqkit`にはほかにも様々な機能があります。

FASTQファイルからFASTAファイルへの変換

In [ ]:
!seqkit fq2fa sample.fastq.gz > sample.fasta

FASTQファイルの統計値をみる

In [ ]:
!seqkit stats sample.fastq.gz

FASTAファイルから指定の長さ以上の配列だけを取り出す

In [ ]:
!seqkit seq -m 10000 sample.fasta > over10000.fasta

FASTAファイルの統計値をみる

In [ ]:
!seqkit stats -a over10000.fasta

配列をreverse complementaryする

In [ ]:
!seqkit seq -pr sample.fasta> reverse_complementary.fasta

---
## まとめ・補足

　ここでは`seqkit`と`blast+`を使って基本的なFASTAファイルの扱い方を試しました。

他にも配列データを扱うツールやコマンドは多数あり、より効率的にFASTAファイルやFASTQファイルを扱うことが可能になります。

大規模なゲノムデータを扱う場合には特に重要になってくるので、必要になれば勉強してみてください。